#### 사용자 정의 데이터셋(이미지 파일) & 커스텀 데이터셋

- 딥러닝은 대량의 데이터를 이용하여 모델을 학습시킨다.
- 데이터를 한번에 메모리에 불러와서 훈련시키면 시간과 비용 측면에서 비효울적이다.
- 커스텀 데이터셋(custom dataset) : 데이터를 한 번에 다 부리지 않고 조금씩 나누어 불러서 사용하는 방식
- 딥러닝 파이토치 교과서, 서지영 지음(p. 46)

In [1]:
# 이미지 파일을 이용하여 사용자 정의 데이터셋 만들기
# 커스텀 데이터셋 클래스를 구현하기 위해 다음 3개의 함수를 구현한다.
# __init__, __len__, __getitem__

import os 
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms

# import torch
# from torchvision import datasets
# from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [2]:
transform_crop = transforms.CenterCrop((64, 64))   # 지정된 사이즈로 가운데 부분만 크롭
# 사용하는 데이터셋의 이미지의 사이즈가 서로 달라, Dataload 부분에서 에러 발생
# 커스텀 데이터셋 클래스의 메서드에서 전달하는 이미지를 미리 크롭하여 반환하도록 코드 설정(250318)

class CustomImageDataset (Dataset):
    # 라벨 정의 파일 csv, 이미지 저장 폴더
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])   # 첫번째 컬럼 : 파일명 , 두번째 컬럼 : 라벨
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return transform_crop(image), label    # 이미지 텐서 + 라벨 반환
        # 가운데 크롭된 이미지를 전달 (250318)

In [3]:
# 전달 변수명 일치시킬 것
custom_img_dataset = CustomImageDataset(annotations_file='./img_labels/annotations_file.csv', img_dir='./img_dir')

In [4]:
# 텐서 출력
# enumerate 메서드 : 갯수 + 추출 데이터 반환

for i, sample in enumerate(custom_img_dataset):
    print(i, sample)

0 (tensor([[[188, 164, 175,  ..., 186, 181, 201],
         [190, 175, 176,  ..., 185, 198, 193],
         [194, 187, 180,  ..., 195, 199, 194],
         ...,
         [171, 167, 146,  ..., 186, 201, 208],
         [168, 149, 163,  ..., 189, 177, 192],
         [152, 154, 159,  ..., 190, 151, 184]],

        [[159, 135, 143,  ..., 156, 151, 169],
         [162, 145, 147,  ..., 158, 170, 161],
         [164, 157, 150,  ..., 167, 171, 162],
         ...,
         [155, 152, 131,  ..., 167, 182, 192],
         [152, 134, 148,  ..., 170, 158, 176],
         [136, 139, 144,  ..., 171, 135, 168]],

        [[117,  95, 102,  ..., 104, 101, 122],
         [123, 107, 107,  ..., 105, 120, 114],
         [128, 121, 112,  ..., 117, 121, 115],
         ...,
         [122, 121, 100,  ..., 135, 150, 158],
         [119, 103, 117,  ..., 138, 126, 142],
         [103, 108, 113,  ..., 141, 102, 135]]], dtype=torch.uint8), 'dog')
1 (tensor([[[215, 215, 211,  ..., 253, 244, 232],
         [225, 238, 246,  

In [ ]:
'''

# 텐서 -> 실제 이미지 변환
tf_img = transforms.ToPILImage()
img = tf_img(sample[0])
img.show()

'''


'\n\n# 텐서 -> 실제 이미지 변환\ntf_img = transforms.ToPILImage()\nimg = tf_img(sample[0])\nimg.show()\n\n'

: 

In [ ]:
# 이미지와 라벨 표시
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(custom_img_dataset, batch_size=16, shuffle=True)
train_dataloader = DataLoader(custom_img_dataset, batch_size=3, shuffle=True)

# data_iter = iter(train_dataloader)
# train_feature, train_label = data_iter.next()
train_feature, train_label = next(iter(train_dataloader))

# print(f"Feature batch shape: {train_feature.size()}")
# print(f"Labels batch shape: {train_label.size()}")

img = train_feature[1].squeeze()

# squeeze() 메서드의 기능 : [[1, 2, 3, 4]]를 [1, 2, 3, 4]의 형태도 변경
label = train_label[1]

# plt.imshow(img, cmap="gray")
# height x width x channel 순으로 저장되어야 이미지 출력 가능

plt.imshow(img.permute(1, 2, 0))   # 채널의 순서를 임의로 조정한다.
plt.show()

print(f"Label: {label}")

# 에러 문제 : 해당 에러는 사용하는 데이터셋의 이미지의 사이즈가 서로 달라서입니다. 
# 사이즈가 다르게되면 Array나 Tensor의 각 차원이 동일하지 않기 때문에 batch형태로 묶어줄 수 없기 때문에 발생합니다.
# https://cchhoo407.tistory.com/32

In [ ]:
# 파이토치 이미지 처리: https://hands-on.pytorch.kr/object-detection/torchvision-basic-transforms.html